In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
input_path = 'output_data/cities.csv'
weather_df = pd.read_csv(input_path)
weather_df = weather_df.drop(columns="Unnamed: 0")
weather_df.head(10)

,Cities,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tuatapere,-46.1333,167.6833,57.90,96.0,100.0,3.87,NZ,1.635135e+09
1,pilar,-9.5972,-35.9567,75.13,91.0,38.0,4.76,BR,1.635135e+09
2,hermanus,-34.4187,19.2345,52.90,80.0,0.0,9.42,ZA,1.635135e+09
3,harper,4.3750,-7.7169,77.27,84.0,100.0,7.49,LR,1.635134e+09
4,hilo,19.7297,-155.0900,73.80,91.0,90.0,5.75,US,1.635135e+09
5,pevek,69.7008,170.3133,6.71,93.0,100.0,5.17,RU,1.635135e+09
6,kapaa,22.0752,-159.3190,80.58,88.0,75.0,1.01,US,1.635135e+09
7,sulphur,30.2366,-93.3774,73.38,97.0,1.0,5.75,US,1.635135e+09
8,tasiilaq,65.6145,-37.6368,30.36,85.0,100.0,47.53,GL,1.635135e+09
9,meulaboh,4.1363,96.1285,83.10,69.0,89.0,2.93,ID,1.635135e+09


## Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
location = weather_df[['Lat', 'Lng']]
humidity = weather_df.Humidity

gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, point_radius=10)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Data Frame with ideal weather

In [5]:
#filter out the ideal weather condition,
#temp no higher than 85, and no lower than 68
# humidity is in range of 65% to 85%
# and, not too windy, wind speed less than 12 mph

ideal_weather_df = weather_df.loc[(weather_df['Max Temp'] <= 85.0) & (weather_df['Max Temp'] >= 68.0)
                                  & (weather_df.Humidity > 30) & (weather_df.Humidity < 60)
                                  & (weather_df['Wind Speed'] < 12)] 
ideal_weather_df

,Cities,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,morwa,22.9000,73.8333,79.12,53.0,0.0,3.22,IN,1.635135e+09
117,mehkar,20.1508,76.5690,76.28,54.0,3.0,6.29,IN,1.635135e+09
130,sur,22.5667,59.5289,84.52,51.0,7.0,9.22,OM,1.635135e+09
155,flinders,-34.5833,150.8552,74.80,57.0,10.0,9.98,AU,1.635135e+09
170,umarkhed,19.6000,77.7000,76.55,59.0,48.0,7.49,IN,1.635135e+09
197,dayong,25.0209,118.2898,75.88,55.0,42.0,4.29,CN,1.635135e+09
272,airai,-8.9266,125.4092,81.12,39.0,93.0,2.39,TL,1.635135e+09
296,bushehr,28.9684,50.8385,82.42,56.0,0.0,9.22,IR,1.635135e+09
297,keti bandar,24.1447,67.4497,81.64,43.0,0.0,4.63,PK,1.635135e+09
315,salalah,17.0151,54.0924,77.09,53.0,0.0,2.30,OM,1.635135e+09


## Hotel Map

In [6]:
hotel_df = pd.DataFrame()
hotel_df['Lat'] = ideal_weather_df['Lat']
hotel_df['Lng'] = ideal_weather_df['Lng']
hotel_df['Hotel Name'] = ''
hotel_df['City'] = ideal_weather_df['Cities']
hotel_df['Country'] = ideal_weather_df['Country']

In [7]:
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
    keyword = "hotels"
    coord_lat = row['Lat']
    coord_lng = row['Lng']
    coord = f'{coord_lat},{coord_lng}'
    radius = 5000
    params = {"location": coord,
             "keyword" : keyword,
             "radius": radius,
             "key": g_key}
    
    hotel_result = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_result['results'][0]['name']
    except:
        print("there is an error")

there is an error


In [124]:
hotel_df

,Lat,Lng,Hotel Name,City,Country
70,22.9000,73.8333,Jay Abe Hotel,morwa,IN
117,20.1508,76.5690,Hotel KV Pride,mehkar,IN
130,22.5667,59.5289,Sur Plaza Hotel,sur,OM
155,-34.5833,150.8552,Lakeview Hotel Motel,flinders,AU
170,19.6000,77.7000,Hotel Sai Atharwa,umarkhed,IN
197,25.0209,118.2898,Yongchang Hotel,dayong,CN
272,-8.9266,125.4092,,airai,TL
296,28.9684,50.8385,Sky 2 hotel,bushehr,IR
297,24.1447,67.4497,Pathan Tea Hotel,keti bandar,PK
315,17.0151,54.0924,Salalah Gardens Hotel,salalah,OM


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))